# 🎤 Reconnaissance Vocale Hors Ligne - Guide d'Utilisation

## 📋 Comment utiliser ce notebook

### Étape 1️⃣ : Installation et Imports (Cellule 1)
Exécutez cette cellule pour importer les bibliothèques nécessaires.
Si les packages ne sont pas installés, décommentez la ligne `!pip install ...` et exécutez-la d'abord.

### Étape 2️⃣ : Configuration (Cellule 2)
Exécutez cette cellule pour charger les configurations des modèles disponibles.

### Étape 3️⃣ : Sélection du Moteur (Cellule 3) ⚠️ **IMPORTANT**
**C'est ici que vous choisissez votre moteur de reconnaissance !**
- Sélectionnez **Sphinx** (intégré, aucun téléchargement) ou **Vosk** (meilleure précision)
- Si vous choisissez Vosk, sélectionnez le modèle de langue
- Cliquez sur "Apply & Load Model"
- **Attendez que le modèle se charge avant de continuer**

### Étape 4️⃣ : Fonctions Utilitaires (Cellule 4)
Exécutez cette cellule pour charger les fonctions de reconnaissance.

### Étape 5️⃣ : Vérifier vos Microphones (Cellule 5)
Optionnel - liste tous vos microphones disponibles.

### Étape 6️⃣ : Test Rapide (Cellule 6)
Décommentez `test_single_recognition()` pour tester avec une seule phrase.

### Étape 7️⃣ : Écoute Continue (Cellule 7)
Décommentez `start_continuous_listening()` pour une reconnaissance en continu.
**Appuyez sur le bouton stop (■) pour arrêter.**

### Étape 8️⃣ : Reconnaissance depuis un Fichier (Cellule 8)
Pour transcrire un fichier audio existant (WAV, AIFF, FLAC).

### Étape 9️⃣ : Créer un Enregistrement de Test (Cellule 9)
Pour créer un fichier audio de test.

---

## 🚀 Démarrage Rapide
1. Exécutez les cellules 1 → 2 → 3 → 4
2. Dans la cellule 3, choisissez votre moteur et cliquez sur "Apply & Load Model"
3. Choisissez une cellule d'utilisation (6, 7, 8, ou 9) et décommentez la fonction
4. Exécutez et parlez !

---

# Cell 1: Installation & Imports

Run each cell sequentially. Start by installing dependencies.


In [14]:
!pip install SpeechRecognition pyaudio pydub pocketsphinx vosk ipywidgets

import speech_recognition as sr
import os
from pathlib import Path
import json
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

print("✓ Imports successful")
print(f"Speech Recognition version: {sr.__version__}")


✓ Imports successful
Speech Recognition version: 3.14.4


Could not find platform independent libraries <prefix>

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# Cell 2: Configuration Setup

Configure your speech recognition settings


In [15]:
# Available Vosk models
VOSK_MODELS = {
    'english_small': {
        'url': 'https://alphacephei.com/vosk/models/vosk-model-small-en-us-0.15.zip',
        'name': 'vosk-model-small-en-us-0.15',
        'size': '40 MB',
        'language': 'English (US)'
    },
    'english_large': {
        'url': 'https://alphacephei.com/vosk/models/vosk-model-en-us-0.22.zip',
        'name': 'vosk-model-en-us-0.22',
        'size': '1.8 GB',
        'language': 'English (US)'
    },
    'french_small': {
        'url': 'https://alphacephei.com/vosk/models/vosk-model-small-fr-0.22.zip',
        'name': 'vosk-model-small-fr-0.22',
        'size': '41 MB',
        'language': 'French'
    },
    'french': {
        'url': 'https://alphacephei.com/vosk/models/vosk-model-fr-0.22.zip',
        'name': 'vosk-model-fr-0.22',
        'size': '1.5 GB',
        'language': 'French'
    }
}

# Global state
class Config:
    engine = 'sphinx'
    vosk_model = None
    vosk_model_name = 'french_small'
    recognizer = sr.Recognizer()

config = Config()

print("✓ Configuration loaded")

✓ Configuration loaded


# Cell 3: Interactive Engine Selection

Select your speech recognition engine


In [25]:
# Create widgets
engine_selector = widgets.RadioButtons(
    options=[
        ('CMU Sphinx (Built-in, no download needed)', 'sphinx'),
        ('Vosk (Better accuracy, requires model download)', 'vosk')
    ],
    value='sphinx',
    description='Engine:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

vosk_model_selector = widgets.Dropdown(
    options=[
        ('🇬🇧 English Small - 40 MB', 'english_small'),
        ('🇬🇧 English Large - 1.8 GB (best accuracy)', 'english_large'),
        ('🇫🇷 French Small - 41 MB', 'french_small'),
        ('🇫🇷 French Large - 1.5 GB (best accuracy)', 'french')
    ],
    value='french_small',
    description='Vosk Model:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px'),
    disabled=True
)

status_output = widgets.Output()
apply_btn = widgets.Button(
    description='Apply & Load Model',
    button_style='success',
    icon='check',
    layout=widgets.Layout(width='200px')
)

# Widget interactions
def on_engine_change(change):
    vosk_model_selector.disabled = (change['new'] == 'sphinx')

def download_and_extract_model(model_info, model_path):
    """Download and extract Vosk model"""
    import urllib.request
    import zipfile
    import shutil

    zip_filename = f"{model_info['name']}.zip"

    print(f"\n📥 Downloading {model_info['name']}...")
    print(f"   Size: {model_info['size']} - This may take a while...")

    try:
        # Download with progress
        def download_progress(block_num, block_size, total_size):
            downloaded = block_num * block_size
            percent = min(downloaded * 100 / total_size, 100)
            print(f"\r   Progress: {percent:.1f}%", end='')

        urllib.request.urlretrieve(
            model_info['url'],
            zip_filename,
            reporthook=download_progress
        )
        print("\n✓ Download complete!")

        # Extract
        print(f"\n📦 Extracting {zip_filename}...")
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall('.')
        print("✓ Extraction complete!")

        # Clean up zip file
        os.remove(zip_filename)
        print("✓ Cleanup complete!")

        return True

    except Exception as e:
        print(f"\n❌ Download/extraction failed: {e}")
        if os.path.exists(zip_filename):
            os.remove(zip_filename)
        return False

def on_apply_click(b):
    config.engine = engine_selector.value
    config.vosk_model_name = vosk_model_selector.value

    with status_output:
        clear_output()
        print(f"✓ Engine set to: {config.engine.upper()}")

        if config.engine == 'vosk':
            model_info = VOSK_MODELS[config.vosk_model_name]
            print(f"\nModel: {model_info['name']}")
            print(f"Language: {model_info['language']}")
            print(f"Size: {model_info['size']}")

            try:
                from vosk import Model
                model_path = Path(model_info['name'])

                if not model_path.exists():
                    print(f"\n⚠️  Model not found locally!")
                    print(f"\n📥 Do you want to download {model_info['name']} ({model_info['size']})?")

                    # Create download buttons
                    download_btn = widgets.Button(
                        description=f'✓ Download ({model_info["size"]})',
                        button_style='success',
                        icon='download'
                    )
                    cancel_btn = widgets.Button(
                        description='✗ Cancel',
                        button_style='danger',
                        icon='times'
                    )

                    button_output = widgets.Output()

                    def on_download(b):
                        with button_output:
                            clear_output()
                            if download_and_extract_model(model_info, model_path):
                                print(f"\n⏳ Loading model...")
                                config.vosk_model = Model(str(model_path))
                                print(f"✓ Model loaded successfully!")
                            else:
                                print("\n❌ Failed to download model. Please try manual download:")
                                print(f"   wget {model_info['url']}")
                                print(f"   unzip {model_info['name']}.zip")
                                config.vosk_model = None

                    def on_cancel(b):
                        with button_output:
                            clear_output()
                            print("\n❌ Download cancelled.")
                            print(f"\nManual download instructions:")
                            print(f"   wget {model_info['url']}")
                            print(f"   unzip {model_info['name']}.zip")
                            print(f"\nOr download from: {model_info['url']}")
                            config.vosk_model = None

                    download_btn.on_click(on_download)
                    cancel_btn.on_click(on_cancel)

                    display(widgets.HBox([download_btn, cancel_btn]))
                    display(button_output)

                else:
                    print(f"\n⏳ Loading model...")
                    config.vosk_model = Model(str(model_path))
                    print(f"✓ Model loaded successfully!")

            except ImportError:
                print(f"\n⚠️  Vosk not installed!")
                print(f"Run: !pip install vosk")
                config.vosk_model = None
            except Exception as e:
                print(f"\n❌ Error: {e}")
                config.vosk_model = None
        else:
            print("\n✓ Using CMU Sphinx (built-in)")
            print("No additional download needed!")

engine_selector.observe(on_engine_change, names='value')
apply_btn.on_click(on_apply_click)

# Display
display(HTML("<h3>🎤 Speech Recognition Configuration</h3>"))
display(widgets.VBox([
    engine_selector,
    vosk_model_selector,
    apply_btn,
    status_output
]))

# Cell 4: Helper Functions

Core recognition functions

In [36]:
def recognize_audio(audio_data):
    """Recognize speech using the configured engine"""

    if config.engine == 'sphinx':
        try:
            return config.recognizer.recognize_sphinx(audio_data)
        except sr.UnknownValueError:
            return None
        except sr.RequestError as e:
            print(f"Sphinx error: {e}")
            return None

    elif config.engine == 'vosk':
        if config.vosk_model is None:
            print("⚠️  Vosk model not loaded. Please configure and load a model first.")
            return None

        try:
            from vosk import KaldiRecognizer
            import io
            import wave

            # Get raw audio data
            wav_data = audio_data.get_wav_data()

            # Read the WAV data to get actual sample rate
            with io.BytesIO(wav_data) as wav_io:
                with wave.open(wav_io, 'rb') as wf:
                    sample_rate = wf.getframerate()
                    frames = wf.readframes(wf.getnframes())

            # Create recognizer with the correct sample rate
            rec = KaldiRecognizer(config.vosk_model, sample_rate)
            rec.SetWords(True)

            # Process the audio data
            rec.AcceptWaveform(frames)
            result = json.loads(rec.FinalResult())

            text = result.get('text', '').strip()

            # Debug: show what Vosk actually returned
            if not text:
                print(f"Debug: Vosk result = {result}")

            return text if text else None

        except Exception as e:
            print(f"Vosk error: {e}")
            import traceback
            traceback.print_exc()
            return None

    return None

def configure_recognizer():
    """Optimize recognizer settings"""
    config.recognizer.energy_threshold = 4000
    config.recognizer.dynamic_energy_threshold = True
    config.recognizer.dynamic_energy_adjustment_damping = 0.15
    config.recognizer.dynamic_energy_ratio = 1.5
    config.recognizer.pause_threshold = 0.8
    config.recognizer.phrase_threshold = 0.3
    config.recognizer.non_speaking_duration = 0.5

print("✓ Helper functions loaded")

✓ Helper functions loaded


# Cell 5: List Available Microphones

Check your available audio input devices


In [18]:
"""
Check your available audio input devices
"""

def list_microphones():
    print("Available microphones:")
    for index, name in enumerate(sr.Microphone.list_microphone_names()):
        print(f"  {index}: {name}")

# Run this to see your microphones
list_microphones()

Available microphones:
  0: Mappeur de sons Microsoft - Input
  1: RÃ©seau de microphones (Realtek(
  2: Casque (PowerLocus)
  3: Mappeur de sons Microsoft - Output
  4: Casque (PowerLocus)
  5: Haut-parleurs (Realtek(R) Audio
  6: Pilote de capture audio principal
  7: RÃ©seau de microphones (Realtek(R) Audio)
  8: Casque (PowerLocus)
  9: PÃ©riphÃ©rique audio principal
  10: Casque (PowerLocus)
  11: Haut-parleurs (Realtek(R) Audio)
  12: Haut-parleurs (Realtek(R) Audio)
  13: Casque (PowerLocus)
  14: Casque (PowerLocus)
  15: RÃ©seau de microphones (Realtek(R) Audio)
  16: Casque ()
  17: Speakers (Nahimic Wave Speaker)
  18: RÃ©seau de microphones (Realtek HD Audio Mic Array input)
  19: Headphones 1 (Realtek HD Audio 2nd output with SST)
  20: Headphones 2 (Realtek HD Audio 2nd output with SST)
  21: Haut-parleur du PC (Realtek HD Audio 2nd output with SST)
  22: Mixage stÃ©rÃ©o (Realtek HD Audio Stereo input)
  23: Microphone (Realtek HD Audio Mic input)
  24: Speakers 1 (Realte

# Cell 6: Single Recognition Test
Test recognition with a single phrase
Run this cell and speak when prompted

In [37]:
def test_single_recognition():
    print(f"Using engine: {config.engine.upper()}")

    if config.engine == 'vosk' and config.vosk_model is None:
        print("⚠️  Please load a Vosk model first (Cell 3)")
        return

    with sr.Microphone() as source:
        print("\n⏳ Adjusting for ambient noise...")
        config.recognizer.adjust_for_ambient_noise(source, duration=1)

        print("🎤 Listening... Speak now!")
        audio = config.recognizer.listen(source, timeout=5, phrase_time_limit=10)

        print("⏳ Processing...")
        text = recognize_audio(audio)

        if text:
            print(f"\n✓ Recognized: '{text}'")
        else:
            print("\n❌ Could not understand the audio")

test_single_recognition()

Using engine: VOSK

⏳ Adjusting for ambient noise...
🎤 Listening... Speak now!
⏳ Processing...

✓ Recognized: 'test'


# Cell 7: Continuous Listening Mode
Continuous speech recognition
Press the stop button (■) to interrupt

In [38]:
def start_continuous_listening():
    print(f"=== Continuous Listening ({config.engine.upper()}) ===")

    if config.engine == 'vosk' and config.vosk_model is None:
        print("⚠️  Please load a Vosk model first (Cell 3)")
        return

    print("\n🎤 Speak clearly into your microphone")
    print("🛑 Press the stop button (■) to exit\n")

    configure_recognizer()

    try:
        with sr.Microphone() as source:
            print("⏳ Adjusting for ambient noise...")
            config.recognizer.adjust_for_ambient_noise(source, duration=1)
            print("✓ Ready! Start speaking...\n")

            phrase_count = 0

            while True:
                try:
                    print(f"[Listening for phrase #{phrase_count + 1}...]")
                    audio = config.recognizer.listen(source, timeout=None, phrase_time_limit=10)

                    print("⏳ Processing...")
                    text = recognize_audio(audio)

                    if text:
                        phrase_count += 1
                        print(f"✓ Phrase #{phrase_count}: '{text}'\n")
                    else:
                        print("❌ Could not understand audio\n")

                except KeyboardInterrupt:
                    break
                except Exception as e:
                    print(f"❌ Error: {e}\n")

    except KeyboardInterrupt:
        pass

    print(f"\n🛑 Stopped. Total phrases recognized: {phrase_count}")

start_continuous_listening()

=== Continuous Listening (VOSK) ===

🎤 Speak clearly into your microphone
🛑 Press the stop button (■) to exit

⏳ Adjusting for ambient noise...
✓ Ready! Start speaking...

[Listening for phrase #1...]
⏳ Processing...
✓ Phrase #1: 'est un test en continu'

[Listening for phrase #2...]
⏳ Processing...
✓ Phrase #2: 'si je tente de parler que me diras-tu'

[Listening for phrase #3...]
⏳ Processing...
✓ Phrase #3: 'peut-être que tu nous diras que j'ai façon'

[Listening for phrase #4...]
⏳ Processing...
✓ Phrase #4: 'peut-être que tu diras des conneries'

[Listening for phrase #5...]

🛑 Stopped. Total phrases recognized: 4


# Cell 8: Recognize from Audio File
Transcribe an existing audio file
Supports: WAV, AIFF, FLAC

In [21]:
def recognize_from_file(audio_file_path):
    if not os.path.exists(audio_file_path):
        print(f"❌ File not found: {audio_file_path}")
        return None

    print(f"Using engine: {config.engine.upper()}")

    if config.engine == 'vosk' and config.vosk_model is None:
        print("⚠️  Please load a Vosk model first (Cell 3)")
        return None

    with sr.AudioFile(audio_file_path) as source:
        print(f"⏳ Loading audio from {audio_file_path}...")
        audio = config.recognizer.record(source)

        print("⏳ Processing...")
        text = recognize_audio(audio)

        if text:
            print(f"\n✓ Recognized: '{text}'")
            return text
        else:
            print("\n❌ Could not understand the audio")
            return None

# Example usage:
# recognize_from_file("your_audio_file.wav")

# Cell 9: Create Test Recording
Record audio to a file for testing

In [22]:
def create_test_recording(filename="test_recording.wav", duration=5):
    with sr.Microphone() as source:
        print(f"🔴 Recording for {duration} seconds...")
        config.recognizer.adjust_for_ambient_noise(source)
        audio = config.recognizer.listen(source, timeout=duration, phrase_time_limit=duration)

        with open(filename, "wb") as f:
            f.write(audio.get_wav_data())

        print(f"✓ Recording saved to {filename}")
        return filename

# Uncomment to run:
# create_test_recording()